In [8]:
using ITensors
using ITensorChemistry 
# using Plots 
# using DelimitedFiles 

# Find best r

In [9]:
function create_molecule(m::String, r::Float64, rep::Int64)
    s = []
    for j in 1:rep
        push!(s,Atom(m, j*r , 0.0, 0.0))
    end
    return Molecule(s)
end

create_molecule (generic function with 1 method)

In [10]:
function energy_at_bond(r,N)
  # define molecule geometry
  molecule = create_molecule("H",r,N)
  # build electronic hamiltonian and solve HF
  hf = molecular_orbital_hamiltonian(molecule; basis="sto-3g")
  hamiltonian = hf.hamiltonian
  hartree_fock_state = hf.hartree_fock_state
  hartree_fock_energy = hf.hartree_fock_energy
  # hilbert space
  s = siteinds("Electron", N; conserve_qns=true)
  H = MPO(hamiltonian, s)
  # initialize MPS to HF state
  ψhf = MPS(s, hartree_fock_state)
  # run dmrg
  dmrg_kwargs = (;
    nsweeps=2,
    maxdim=[10,20,30,40,50,100],
    cutoff=1e-8,
    noise=[1e-6, 1e-7, 1e-8, 0.0],
  )
  dmrg_energy, _ = dmrg(H, ψhf; nsweeps=2, outputlevel=0)
  return hartree_fock_energy, dmrg_energy
end

energy_at_bond (generic function with 1 method)

In [6]:
# bond distances
r⃗ = 0.1:0.1:2
hartee_energy = []
dmrg_energy = []
for r in r⃗
    push!(hartee_energy, energy_at_bond(r,2)[1]);
    push!(dmrg_energy, energy_at_bond(r,2)[2]);
end;

converged SCF energy = -2.06919742280387
RHF Energy (Ha): -2.069197422803868
converged SCF energy = -2.06919742280387
RHF Energy (Ha): -2.069197422803868
converged SCF energy = -2.12138675587022
RHF Energy (Ha): -2.121386755870221
converged SCF energy = -2.12138675587022
RHF Energy (Ha): -2.121386755870221
converged SCF energy = -2.12425973897278
RHF Energy (Ha): -2.1242597389727775
converged SCF energy = -2.12425973897278
RHF Energy (Ha): -2.1242597389727766
converged SCF energy = -2.09854593699772
RHF Energy (Ha): -2.098545936997719
converged SCF energy = -2.09854593699772
RHF Energy (Ha): -2.098545936997718


In [7]:
r = r⃗[findmin(dmrg_energy)[2]]

0.9

# H-chain overlap

In [ ]:
@time function create_molecule(m::String, r::Float64, rep::Int64)
    s = []
    for j in 1:rep
        push!(s,Atom(m, j*r , 0.0, 0.0))
    end
    return Molecule(s)
end;

In [106]:
@time Ms = create_molecule("H",0.9,6);

  0.000010 seconds (16 allocations: 688 bytes)


@time function create_Hs(Ms)
    mols = []
    for j in 1:Int(length(Ms.atoms)/2)
        push!(mols,Ms.atoms[1:2j])
    end
    return mols 
end;
@time mols = create_Hs(Ms);

In [91]:
function Hartee_Fock_energy(Ms)
    Hamiltonian = []
    states = []
    HF_energy = []
    mpos = []
    s = siteinds("Electron", length(Ms)+2)
    for j in 1:Int(length(Ms.atoms)/2)
        oh = molecular_orbital_hamiltonian(Molecule(Ms.atoms[1:2j]); basis="sto-3g")
        push!(Hamiltonian, oh.hamiltonian)
        push!(states, oh.hartree_fock_state)
        push!(HF_energy, oh.hartree_fock_energy)
        push!(mpos,MPO(oh.hamiltonian,s[1:2j]))
        
    end
    return Hamiltonian, HF_energy, states
end

Hartee_Fock_energy (generic function with 1 method)

In [94]:
h,e,s = Hartee_Fock_energy(Ms);

converged SCF energy = -1.09191404102006
RHF Energy (Ha): -1.0919140410200572
converged SCF energy = -2.12425973897278
RHF Energy (Ha): -2.1242597389727784
converged SCF energy = -3.16074336356199
RHF Energy (Ha): -3.1607433635619895


In [ ]:
function DMRG_energy()
    s = siteinds("Electron", length(Ms)+2)
    mpos = []
end

In [114]:
a = []
for j in 1:3
    insert!(a,j,j^2)
end
println(a)

Any[1, 4, 9]
